In [44]:
pip install delta-spark

Note: you may need to restart the kernel to use updated packages.


In [45]:
from os import PathLike
from hdfs import InsecureClient
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *
from pyspark.sql.types import *
from pyspark.sql.functions import *

# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/Qualidade_NYC/gold'

builder = SparkSession \
    .builder \
    .master("local[2]") \
    .appName("Python Spark DataFrames and SQL") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .enableHiveSupport() \

spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [46]:
spark.sql(
    """
    CREATE DATABASE IF NOT EXISTS gold LOCATION 'hdfs://hdfs-nn:9000/Qualidade_NYC/gold/Projeto.db/'
    """
)

DataFrame[]

In [47]:
# create Children table in gold

spark.sql(
    """
    DROP TABLE IF EXISTS Projeto.gold_Children
    """
)



DataFrame[]

In [48]:
spark.sql(
    """
    CREATE EXTERNAL TABLE Projeto.gold_Children (
        geo_area_name STRING,
        time_period INT,
        BLL_Bigger_or_equal_5 INT,
        BLL_Bigger_or_equal_10 INT,
        BLL_Bigger_or_equal_15 INT,
        BLL_Number_Tested FLOAT,
        BLL_Bigger_or_equal_15_per_1000_tested FLOAT,
        Percentage_BLL_Bigger_or_equal_5 DOUBLE,
        Percentage_BLL_Bigger_or_equal_10 DOUBLE,
        Percentage_BLL_Bigger_or_equal_15 DOUBLE
    )
    USING DELTA
    LOCATION 'hdfs://hdfs-nn:9000/Qualidade_NYC/gold/projeto.db/gold_Children/'
    """
)

DataFrame[]

In [49]:
from pyspark.sql.functions import substring, avg, sum

# read Children from the silver tables
hdfs_path = "hdfs://hdfs-nn:9000/Qualidade_NYC/silver/projeto.db/Children/"

gold_Children = spark\
             .read\
             .load(hdfs_path)

gold_Children.show()
gold_Children.printSchema()

+--------+-----------+-------------+----------+-----------+--------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------+---------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+
|geo_type|geo_area_id|geo_area_name|borough_id|time_period|Children_under_6_years_with_elevated_blood_lead_levels__Number_BLL__Bigger_or_equal_5_|Children_under_6_years_with_elevated_blood_lead_levels__Number_BLL_Bigger_or_equal_10_|Children_under_6_years_with_elevated_blood_lead_levels__Number_BLL_Bigger

In [50]:
rename=(column.replace('Children_under_6_years_with_elevated_blood_lead_levels__Rate__','') for column in gold_Children.columns)
gold_Children = gold_Children.toDF(*rename)
rename=(column.replace('Children_under_6_years_with_elevated_blood_lead_levels__Rate_','') for column in gold_Children.columns)
gold_Children = gold_Children.toDF(*rename)
rename=(column.replace('Children_under_6_years_with_elevated_blood_lead_levels__Number_','') for column in gold_Children.columns)
gold_Children = gold_Children.toDF(*rename)
rename=(column.replace('Tested','BLL_Number_Tested') for column in gold_Children.columns)
gold_Children = gold_Children.toDF(*rename)
rename=(column.replace('5_','5') for column in gold_Children.columns)
gold_Children = gold_Children.toDF(*rename)
rename=(column.replace('10_','10') for column in gold_Children.columns)
gold_Children = gold_Children.toDF(*rename)
rename=(column.replace('BLL__','BLL_') for column in gold_Children.columns)
gold_Children = gold_Children.toDF(*rename)
gold_Children.toPandas()

,geo_type,geo_area_id,geo_area_name,borough_id,time_period,BLL_Bigger_or_equal_5,BLL_Bigger_or_equal_10,BLL_Bigger_or_equal_15,BLL_Number_Tested,BLL_Bigger_or_equal_5_per_1000_tested,BLL_Bigger_or_equal_10_per_1000_tested,BLL_Bigger_or_equal_15_per_1000_tested
0,Borough,1,Bronx,1.0,2005,8245,595,167,64500.0,127.699997,9.2,2.6
1,Borough,1,Bronx,1.0,2006,7272,474,144,67200.0,108.199997,7.1,2.1
2,Borough,1,Bronx,1.0,2007,6174,438,135,68300.0,90.400002,6.4,2.0
3,Borough,1,Bronx,1.0,2008,4254,292,105,69800.0,60.900002,4.2,1.5
4,Borough,1,Bronx,1.0,2009,2742,278,103,70000.0,39.200001,4.0,1.5
...,...,...,...,...,...,...,...,...,...,...,...,...
1147,Neighborhood (UHF 42),104,Pelham - Throgs Neck,1.0,2005,1314,86,24,11600.0,113.199997,7.4,2.1
1148,Neighborhood (UHF 42),301,Washington Heights,3.0,2006,1115,104,36,11200.0,99.800003,9.3,3.2
1149,Neighborhood (UHF 42),407,Southwest Queens,4.0,2013,239,39,15,11400.0,20.900000,3.4,1.3
1150,Neighborhood (UHF 42),106,High Bridge - Morrisania,1.0,2013,281,31,9,11800.0,23.799999,2.6,0.8


In [51]:
gold_Children = gold_Children.drop("geo_type")
gold_Children = gold_Children.drop("geo_area_id")
gold_Children = gold_Children.drop("borough_id")
gold_Children = gold_Children.drop("BLL_Bigger_or_equal_5_per_1000_tested")
gold_Children = gold_Children.drop("BLL_Bigger_or_equal_10_per_1000_tested")

In [52]:
gold_Children = gold_Children.withColumn('Percentage_BLL_Bigger_or_equal_5',expr("(BLL_Bigger_or_equal_5/BLL_Number_Tested)*100"))
gold_Children = gold_Children.withColumn('Percentage_BLL_Bigger_or_equal_10',expr("(BLL_Bigger_or_equal_10/BLL_Number_Tested)*100"))
gold_Children = gold_Children.withColumn('Percentage_BLL_Bigger_or_equal_15',expr("(BLL_Bigger_or_equal_15/BLL_Number_Tested)*100"))
gold_Children.printSchema()
gold_Children.show()

root
 |-- geo_area_name: string (nullable = true)
 |-- time_period: integer (nullable = true)
 |-- BLL_Bigger_or_equal_5: integer (nullable = true)
 |-- BLL_Bigger_or_equal_10: integer (nullable = true)
 |-- BLL_Bigger_or_equal_15: integer (nullable = true)
 |-- BLL_Number_Tested: float (nullable = true)
 |-- BLL_Bigger_or_equal_15_per_1000_tested: float (nullable = true)
 |-- Percentage_BLL_Bigger_or_equal_5: double (nullable = true)
 |-- Percentage_BLL_Bigger_or_equal_10: double (nullable = true)
 |-- Percentage_BLL_Bigger_or_equal_15: double (nullable = true)

+-------------+-----------+---------------------+----------------------+----------------------+-----------------+--------------------------------------+--------------------------------+---------------------------------+---------------------------------+
|geo_area_name|time_period|BLL_Bigger_or_equal_5|BLL_Bigger_or_equal_10|BLL_Bigger_or_equal_15|BLL_Number_Tested|BLL_Bigger_or_equal_15_per_1000_tested|Percentage_BLL_Bigger_or

In [53]:
#group by name, geo place name
from pyspark.sql.functions import sum,avg,max,count
gold = gold_Children.groupBy("geo_area_name").count()\
    .show()

+--------------------+-----+
|       geo_area_name|count|
+--------------------+-----+
|     Northeast Bronx|   24|
|Greenwich Village...|   24|
|              Queens|   24|
|     Lower Manhattan|   24|
|Central Harlem - ...|   24|
|Stapleton - St. G...|   24|
|Union Square - Lo...|   24|
|  Washington Heights|   24|
|    Southeast Queens|   24|
|East Flatbush - F...|   24|
|Ridgewood - Fores...|   24|
|     Upper East Side|   24|
|Hunts Point - Mot...|   24|
|Gramercy Park - M...|   24|
|         East Harlem|   24|
|    Southwest Queens|   24|
|         Sunset Park|   24|
|Bensonhurst - Bay...|   24|
|Bedford Stuyvesan...|   24|
|Downtown - Height...|   24|
+--------------------+-----+
only showing top 20 rows



In [54]:
# write to delta table
gold_Children \
    .select("geo_area_name","time_period","BLL_Bigger_or_equal_5","BLL_Bigger_or_equal_10","BLL_Bigger_or_equal_15","BLL_Number_Tested","BLL_Bigger_or_equal_15_per_1000_tested","Percentage_BLL_Bigger_or_equal_5","Percentage_BLL_Bigger_or_equal_10","Percentage_BLL_Bigger_or_equal_15") \
    .write \
    .format("delta") \
    .mode("overwrite") \
    .save("hdfs://hdfs-nn:9000/Qualidade_NYC/gold/Projeto.db/gold_Children/")

In [55]:
# check the results in the table
spark.table("Projeto.gold_Children").show()
spark.table("Projeto.gold_Children").count()

+-------------+-----------+---------------------+----------------------+----------------------+-----------------+--------------------------------------+--------------------------------+---------------------------------+---------------------------------+
|geo_area_name|time_period|BLL_Bigger_or_equal_5|BLL_Bigger_or_equal_10|BLL_Bigger_or_equal_15|BLL_Number_Tested|BLL_Bigger_or_equal_15_per_1000_tested|Percentage_BLL_Bigger_or_equal_5|Percentage_BLL_Bigger_or_equal_10|Percentage_BLL_Bigger_or_equal_15|
+-------------+-----------+---------------------+----------------------+----------------------+-----------------+--------------------------------------+--------------------------------+---------------------------------+---------------------------------+
+-------------+-----------+---------------------+----------------------+----------------------+-----------------+--------------------------------------+--------------------------------+---------------------------------+-------------------

0

In [56]:
spark.sql("""
GENERATE symlink_format_manifest FOR TABLE delta.`hdfs://hdfs-nn:9000/Qualidade_NYC/gold/Projeto.db/gold_Children/`
""").show()

++
||
++
++



In [57]:
spark.sql("""
DROP TABLE IF EXISTS Projeto.gold_Children_presto 
""").show()


++
||
++
++



In [58]:

spark.sql("""
    CREATE EXTERNAL TABLE Projeto.gold_Children_presto (
        geo_area_name STRING,
        time_period INT,
        BLL_Bigger_or_equal_5 INT,
        BLL_Bigger_or_equal_10 INT,
        BLL_Bigger_or_equal_15 INT,
        BLL_Number_Tested FLOAT,
        BLL_Bigger_or_equal_15_per_1000_tested FLOAT,
        Percentage_BLL_Bigger_or_equal_5 DOUBLE,
        Percentage_BLL_Bigger_or_equal_10 DOUBLE,
        Percentage_BLL_Bigger_or_equal_15 DOUBLE
    )
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
STORED AS INPUTFORMAT 'org.apache.hadoop.hive.ql.io.SymlinkTextInputFormat'
OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION 'hdfs://hdfs-nn:9000/Qualidade_NYC/gold/Projeto.db/gold_Children/_symlink_format_manifest'
""").show()

++
||
++
++

